My goal with this jupiter note book is to combine all my work and add the latest asked changes to collect the std and moy for each predicted sample using MC dropout BNN aproximation and Bayes by back propagation.

LOAD DATA: I am loading the same dataset that I will be using for training both models later, which was before collected using revound simulation.

In [9]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# 1. Load CSV
def load_data(csv_path, label_col='status', coord_cols=['a', 'e', 'i']):
    df = pd.read_csv(csv_path)
    # Map label 2 to 1 if needed
    df[label_col] = df[label_col].replace(2, 1)
    # Extract inputs and labels
    X = torch.tensor(df[coord_cols].values, dtype=torch.float32)
    y = torch.tensor(df[label_col].values, dtype=torch.long)
    return X, y

X , y = load_data(r"C:\Users\User\Desktop\PROJECTS\stage\project_submession\combined_dataset.csv")

# # 2. Separate features and labels
# X = data[['a', 'e', 'i']].values  # shape (N,3)
# y = data['status'].values           # shape (N,)

# 3. Train/test split (optional but good practice)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # long for classification

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Prepare DataLoader (for batching)
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


C:\Users\User\AppData\Local\Temp\ipykernel_32316\3582689913.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
C:\Users\User\AppData\Local\Temp\ipykernel_32316\3582689913.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # long for classification
C:\Users\User\AppData\Local\Temp\ipykernel_32316\3582689913.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
C:\

mc dropout

In [4]:
#---------------------------------Mc dropout takes (a e i) as an input and gave the classification and the prob using softmax function-------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F

class MCDropoutModel(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=2, dropout_rate=0.5):
        super(MCDropoutModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        self.fc3 = nn.Linear(hidden_dim, output_dim)  # output_dim=2 for binary classification (logits)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)  # dropout always active during both training and inference for MC Dropout
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)       # raw logits (before softmax)
        return x

    def predict_mc_dropout(self, x, n_samples=100):
       self.train()  # dropout active
       preds = []
       for _ in range(n_samples):
           logits = self.forward(x)  # shape: (batch_size, output_dim)
           prob = torch.softmax(logits, dim=1)  # shape: (batch_size, output_dim)
           preds.append(prob.unsqueeze(0))  # add sample dim at dim 0
       preds = torch.cat(preds, dim=0)  # shape: (n_samples, batch_size, output_dim)
       return preds


# Instantiate model
model = MCDropoutModel(input_dim=3, hidden_dim=64, output_dim=2, dropout_rate=0.5)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define loss function
criterion = nn.CrossEntropyLoss()


In [ ]:
#-------------------------------------------------------model training and saving------------------------------------------------------------------------------- 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 50  

for epoch in range(num_epochs):
    model.train()  # set model to training mode (dropout active)
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)  # forward pass
        loss = criterion(outputs, labels)  # compute loss
        
        loss.backward()  # backpropagation
        optimizer.step()  # update weights
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")

# save the model weights
torch.save(model.state_dict(), "mc_dropout_BNN.pth") 
print("Model saved successfully.")


Epoch 1/50 - Loss: 1.6611
Epoch 2/50 - Loss: 0.7325
Epoch 3/50 - Loss: 0.5873
Epoch 4/50 - Loss: 0.5229
Epoch 5/50 - Loss: 0.4829
Epoch 6/50 - Loss: 0.4470
Epoch 7/50 - Loss: 0.4073
Epoch 8/50 - Loss: 0.3829
Epoch 9/50 - Loss: 0.3487
Epoch 10/50 - Loss: 0.3275
Epoch 11/50 - Loss: 0.3021
Epoch 12/50 - Loss: 0.2753
Epoch 13/50 - Loss: 0.2611
Epoch 14/50 - Loss: 0.2343
Epoch 15/50 - Loss: 0.2189
Epoch 16/50 - Loss: 0.2089
Epoch 17/50 - Loss: 0.1940
Epoch 18/50 - Loss: 0.1787
Epoch 19/50 - Loss: 0.1753
Epoch 20/50 - Loss: 0.1750
Epoch 21/50 - Loss: 0.1615
Epoch 22/50 - Loss: 0.1466
Epoch 23/50 - Loss: 0.1478
Epoch 24/50 - Loss: 0.1401
Epoch 25/50 - Loss: 0.1283
Epoch 26/50 - Loss: 0.1250
Epoch 27/50 - Loss: 0.1292
Epoch 28/50 - Loss: 0.1222
Epoch 29/50 - Loss: 0.1277
Epoch 30/50 - Loss: 0.1145
Epoch 31/50 - Loss: 0.1165
Epoch 32/50 - Loss: 0.1166
Epoch 33/50 - Loss: 0.1019
Epoch 34/50 - Loss: 0.1096
Epoch 35/50 - Loss: 0.1036
Epoch 36/50 - Loss: 0.0977
Epoch 37/50 - Loss: 0.1008
Epoch 38/5

In [6]:
#-------------------------------------------------inference function-----------------------------------------------------------------------------
def run_inference_with_uncertainty(model, data_df, device, n_samples=100, filter_threshold=30):
    """
    Runs MC Dropout inference on a filtered DataFrame and returns results with uncertainty measures.
    
    Args:
        model: Trained MCDropoutModel
        data_df: Input DataFrame with columns ['a','e','i'] and optionally 'name'
        device: torch.device
        n_samples: Number of MC Dropout forward passes
        filter_threshold: Keep rows where a > threshold
    
    Returns:
        DataFrame with predictions and uncertainty statistics
    """
    # Ensure identifier column
    if 'name' not in data_df.columns:
        data_df = data_df.reset_index().rename(columns={'index': 'name'})

    # Filter rows
    filtered_data = data_df[data_df['a'] > filter_threshold].copy()
    if filtered_data.empty:
        print("⚠️ No rows passed the filter condition.")
        return filtered_data

    # Prepare input tensor
    X_filtered = filtered_data[['a', 'e', 'i']].values
    X_filtered_tensor = torch.tensor(X_filtered, dtype=torch.float32).to(device)

    # Run MC Dropout inference
    with torch.no_grad():
        preds = model.predict_mc_dropout(X_filtered_tensor, n_samples=n_samples)  # (n_samples, batch, num_classes)

    # Mean & std of probabilities
    mean_preds = preds.mean(dim=0)                                # (batch, num_classes)
    std_preds  = preds.std(dim=0, unbiased=False)                  # (batch, num_classes)

    # Predicted classes
    predicted_classes = torch.argmax(mean_preds, dim=1)            # (batch,)

    # Predicted-class probability mean & std
    gather_index = predicted_classes.unsqueeze(0).unsqueeze(2).expand(preds.size(0), -1, 1)
    preds_for_predclass = preds.gather(dim=2, index=gather_index).squeeze(2)
    predicted_prob_mean = preds_for_predclass.mean(dim=0)
    predicted_prob_std  = preds_for_predclass.std(dim=0, unbiased=False)

    # Predictive entropy
    entropy = -torch.sum(mean_preds * torch.log(mean_preds + 1e-12), dim=1)

    # Variation ratio (using mode)
    mc_pred_classes = torch.argmax(preds, dim=2).cpu().numpy()
    from scipy.stats import mode
    mode_class, mode_count = mode(mc_pred_classes, axis=0)
    variation_ratio = 1 - (mode_count[0] / preds.shape[0])

    # Attach results
    filtered_data['predicted_class'] = predicted_classes.cpu().numpy()
    filtered_data['predicted_prob_mean'] = predicted_prob_mean.cpu().numpy()
    filtered_data['predicted_prob_std']  = predicted_prob_std.cpu().numpy()
    filtered_data['predictive_entropy']  = entropy.cpu().numpy()
    filtered_data['variation_ratio']     = variation_ratio.astype(float)

    # Also keep per-class mean & std
    num_classes = mean_preds.shape[1]
    for c in range(num_classes):
        filtered_data[f'prob_mean_c{c}'] = mean_preds[:, c].cpu().numpy()
        filtered_data[f'prob_std_c{c}']  = std_preds[:, c].cpu().numpy()

    return filtered_data


In [7]:
def save_inference_results(df, summary_path="summary_orbital_predictions.csv", detailed_path=None):
    """
    Saves inference results DataFrame to CSVs (summary + optional detailed).
    """
    # Summary columns
    selected_columns = [
        'name', 'a', 'e', 'i',
        'predicted_class',
        'predicted_prob_mean', 'predicted_prob_std',
        'predictive_entropy', 'variation_ratio'
    ]
    # Include per-class mean & std if present
    for col in df.columns:
        if col.startswith("prob_mean_c") or col.startswith("prob_std_c"):
            selected_columns.append(col)

    summary_df = df[selected_columns].copy()
    summary_df.to_csv(summary_path, index=False)
    print(f"✅ Saved summary to '{summary_path}'")

    # Save full DataFrame if requested
    if detailed_path:
        df.to_csv(detailed_path, index=False)
        print(f"✅ Saved detailed results to '{detailed_path}'")


Now after having our model trained and inference function ready we can proceed with our MPC data 

In [8]:
# Load test CSV
test_data = pd.read_csv(r"C:\Users\User\Desktop\PROJECTS\stage\converted_data.csv")

# Run inference
results_df = run_inference_with_uncertainty(model, test_data, device, n_samples=100, filter_threshold=30)

# Save CSVs
if not results_df.empty:
    save_inference_results(results_df,
                           summary_path="MC_Dropout_Inference.csv",
                           detailed_path="MC_dropout_Inference_detailed.csv")


✅ Saved summary to 'MC_Dropout_Inference.csv'
✅ Saved detailed results to 'MC_dropout_Inference_detailed.csv'


We can also analyse our model performance using the test_loader data from the synthetic rebound data

In [9]:
# --- Evaluate model on held-out test data (20%) ---
model.eval()  # set to eval mode (dropout OFF for deterministic evaluation)

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)  # logits
        loss = criterion(outputs, labels)

        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Average loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = 100 * correct / total

print(f"📊 Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")


📊 Test Loss: 0.1233 | Test Accuracy: 93.26%


In [11]:
# --- Uncertainty analysis on test set (20%) ---
# Collect test inputs and labels from test_loader
import numpy as np 
all_inputs = []
all_labels = []

for inputs, labels in test_loader:
    all_inputs.append(inputs.cpu().numpy())
    all_labels.append(labels.cpu().numpy())

X_test = np.vstack(all_inputs)
y_test = np.hstack(all_labels)

# Put into DataFrame
test_df = pd.DataFrame(X_test, columns=['a', 'e', 'i'])
test_df['true_label'] = y_test

# Run inference with MC Dropout (keep all test samples, no filtering)
results_test_df = run_inference_with_uncertainty(
    model, test_df, device, n_samples=100, filter_threshold=-np.inf
)

# Save results
save_inference_results(
    results_test_df,
    summary_path="test_summary_predictions.csv",
    detailed_path="test_detailed_predictions.csv"
)


✅ Saved summary to 'test_summary_predictions.csv'
✅ Saved detailed results to 'test_detailed_predictions.csv'


Now we move forward testing the bayes by back propagation

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# ---- Bayesian Linear Layer ----
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_sigma=0.1):
        super(BayesianLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Mean & rho (to compute std via softplus)
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-0.2, 0.2))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-5, -4))

        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5, -4))

        # Prior
        self.prior_sigma = prior_sigma
        self.prior = torch.distributions.Normal(0, prior_sigma)

    def forward(self, x):
        # Compute std from rho
        weight_sigma = torch.log1p(torch.exp(self.weight_rho))
        bias_sigma = torch.log1p(torch.exp(self.bias_rho))

        # Sample weights & bias using reparameterization
        weight_eps = torch.randn_like(weight_sigma)
        bias_eps = torch.randn_like(bias_sigma)

        weight = self.weight_mu + weight_sigma * weight_eps
        bias = self.bias_mu + bias_sigma * bias_eps

        return F.linear(x, weight, bias)

# ---- BBB Neural Network ----
class BBBModel(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=2):
        super(BBBModel, self).__init__()
        self.blinear1 = BayesianLinear(input_dim, hidden_dim)
        self.blinear2 = BayesianLinear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.blinear1(x))
        return self.blinear2(x)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# ---- Bayesian Linear Layer ----
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_sigma=0.1):
        super(BayesianLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Mean & rho (to compute std via softplus)
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-0.2, 0.2))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-5, -4))

        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5, -4))

        # Prior
        self.prior_sigma = prior_sigma
        self.prior = torch.distributions.Normal(0, prior_sigma)

    def forward(self, x):
        # Compute std from rho
        weight_sigma = torch.log1p(torch.exp(self.weight_rho))
        bias_sigma = torch.log1p(torch.exp(self.bias_rho))

        # Sample weights & bias using reparameterization
        weight_eps = torch.randn_like(weight_sigma)
        bias_eps = torch.randn_like(bias_sigma)

        weight = self.weight_mu + weight_sigma * weight_eps
        bias = self.bias_mu + bias_sigma * bias_eps

        return F.linear(x, weight, bias)

# ---- BBB Neural Network ----
class BBBModel(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, output_dim=2):
        super(BBBModel, self).__init__()
        self.blinear1 = BayesianLinear(input_dim, hidden_dim)
        self.blinear2 = BayesianLinear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.blinear1(x))
        return self.blinear2(x) 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def train_bbb(model, train_loader, device, n_epochs=50, lr=1e-3):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)

            # Negative log-likelihood
            loss = F.cross_entropy(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{n_epochs} - Loss: {total_loss/len(train_loader):.4f}")


In [6]:

import numpy as np
import pandas as pd
import torch.nn.functional as F

def run_bbb_inference(model, data_df, device, n_samples=100, filter_threshold=-np.inf):
    model.eval()
    results = []

    X = torch.tensor(data_df[['a', 'e', 'i']].values, dtype=torch.float32).to(device)

    with torch.no_grad():
        for idx in range(len(X)):
            preds = []
            for _ in range(n_samples):
                outputs = model(X[idx].unsqueeze(0))
                probs = F.softmax(outputs, dim=1).cpu().numpy()[0]
                preds.append(probs)

            preds = np.array(preds)
            mean_probs = preds.mean(axis=0)
            std_probs = preds.std(axis=0)

            predicted_class = mean_probs.argmax()
            confidence = mean_probs[predicted_class]
            entropy = -np.sum(mean_probs * np.log(mean_probs + 1e-10))
            variation_ratio = 1 - (np.bincount(preds.argmax(axis=1), minlength=mean_probs.shape[0]).max() / n_samples)

            # Apply filtering if required
            if confidence >= filter_threshold:
                results.append({
                   "sample_index": idx,
                    "predicted_class": predicted_class,
                    "confidence": confidence,
                    "entropy": entropy,
                    "variation_ratio": variation_ratio,
                    "mean_prob": mean_probs.mean(),   # <- unified name
                    "std_prob": std_probs.mean(),     # <- unified name
                    "mean_probs": mean_probs.tolist(), # full distribution
                    "std_probs": std_probs.tolist()})

    return pd.DataFrame(results)

def save_bbb_results(results_df, summary_path="bbb_summary.csv", detailed_path="bbb_detailed.csv"):
    summary = results_df.drop(columns=["mean_probs", "std_probs"], errors="ignore")
    summary.to_csv(summary_path, index=False)
    results_df.to_csv(detailed_path, index=False)
    print(f"✅ Results saved: {summary_path}, {detailed_path}")
 

In [ ]:
# === Usage on MPC orbital data with filtering ===
import pandas as pd
import torch
# Load MPC orbital data
mpc_data = pd.read_csv(r"C:\Users\User\Desktop\PROJECTS\stage\project_submession\converted_data.csv")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BBBModel(input_dim=3, hidden_dim=64, output_dim=2)
trained_model = train_bbb(model, train_loader, device, n_epochs=50, lr=1e-3)



Epoch 1/50 - Loss: 0.6908
Epoch 2/50 - Loss: 0.5566
Epoch 3/50 - Loss: 0.4666
Epoch 4/50 - Loss: 0.3710
Epoch 5/50 - Loss: 0.2985
Epoch 6/50 - Loss: 0.2966
Epoch 7/50 - Loss: 0.2200
Epoch 8/50 - Loss: 0.2086
Epoch 9/50 - Loss: 0.1800
Epoch 10/50 - Loss: 0.1583
Epoch 11/50 - Loss: 0.1352
Epoch 12/50 - Loss: 0.1422
Epoch 13/50 - Loss: 0.1290
Epoch 14/50 - Loss: 0.1185
Epoch 15/50 - Loss: 0.1217
Epoch 16/50 - Loss: 0.1021
Epoch 17/50 - Loss: 0.1065
Epoch 18/50 - Loss: 0.1054
Epoch 19/50 - Loss: 0.0977
Epoch 20/50 - Loss: 0.1008
Epoch 21/50 - Loss: 0.0929
Epoch 22/50 - Loss: 0.0891
Epoch 23/50 - Loss: 0.0844
Epoch 24/50 - Loss: 0.0777
Epoch 25/50 - Loss: 0.0962
Epoch 26/50 - Loss: 0.0778
Epoch 27/50 - Loss: 0.0812
Epoch 28/50 - Loss: 0.0895
Epoch 29/50 - Loss: 0.0778
Epoch 30/50 - Loss: 0.0722
Epoch 31/50 - Loss: 0.0769
Epoch 32/50 - Loss: 0.0772
Epoch 33/50 - Loss: 0.0737
Epoch 34/50 - Loss: 0.0735
Epoch 35/50 - Loss: 0.0758
Epoch 36/50 - Loss: 0.0843
Epoch 37/50 - Loss: 0.0682
Epoch 38/5

In [ ]:

# Run BBB inference (with n_samples MC forward passes)
results_df = run_bbb_inference(
    model,
    filtered_mpc_data,
    device,
    n_samples=100,           # number of stochastic forward passes
    filter_threshold=-np.inf 
)

# Merge results back with orbital parameters
merged_results = pd.concat([filtered_mpc_data.reset_index(drop=True), results_df], axis=1)

# Save results (both summary & detailed versions)
save_bbb_results(
    merged_results,
    summary_path="bbb_summary_mpc.csv",
    detailed_path="bbb_detailed_mpc.csv"
)

print("✅ BBB inference complete with filtering. Saved to 'bbb_summary_mpc.csv' and 'bbb_detailed_mpc.csv'")


✅ Results saved: bbb_summary_mpc.csv, bbb_detailed_mpc.csv
✅ BBB inference complete with filtering. Saved to 'bbb_summary_mpc.csv' and 'bbb_detailed_mpc.csv'


: 

In [17]:
#------------------------------------------a small evaluation ----------------------------------------------
import torch.nn.functional as F
import numpy as np

def evaluate_bbb(model, X_test_tensor, y_test_tensor, device, n_samples=50):
    model.eval()
    X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(device)

    all_mean_preds = []
    all_true = []

    with torch.no_grad():
        for idx in range(len(X_test_tensor)):
            x = X_test_tensor[idx].unsqueeze(0)
            preds = []
            for _ in range(n_samples):
                logits = model(x)
                probs = F.softmax(logits, dim=1)
                preds.append(probs.cpu().numpy()[0])

            preds = np.array(preds)
            mean_probs = preds.mean(axis=0)

            all_mean_preds.append(mean_probs)
            all_true.append(y_test_tensor[idx].item())

    all_mean_preds = np.array(all_mean_preds)
    all_true = np.array(all_true)

    # Predicted classes
    predicted_classes = all_mean_preds.argmax(axis=1)

    # Accuracy
    accuracy = (predicted_classes == all_true).mean()

    # Negative log-likelihood
    nll = -np.mean([np.log(all_mean_preds[i, all_true[i]] + 1e-12) for i in range(len(all_true))])

    print(f"✅ Test Accuracy: {accuracy:.4f}")
    print(f"✅ Test NLL: {nll:.4f}")

    return accuracy, nll, predicted_classes, all_mean_preds, all_true

# Run evaluation
accuracy, nll, preds, probs, labels = evaluate_bbb(model, X_test_tensor, y_test_tensor, device, n_samples=100)


✅ Test Accuracy: 0.9326
✅ Test NLL: 0.1233


In [18]:
#---------------------------------this cell is to save prediction of the splitted test data (synthetic)
# === Evaluate BBB model on test split + save inference results ===
import pandas as pd
import torch.nn.functional as F
import numpy as np
from scipy.stats import mode

def evaluate_and_save_bbb_test(model, X_test_tensor, y_test_tensor, device, n_samples=100,
                               summary_path="bbb_summary_test.csv",
                               detailed_path="bbb_detailed_test.csv"):
    model.eval()
    X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(device)
    
    results = []
    
    with torch.no_grad():
        for idx in range(len(X_test_tensor)):
            x = X_test_tensor[idx].unsqueeze(0)
            preds_mc = []
            for _ in range(n_samples):
                logits = model(x)
                probs = F.softmax(logits, dim=1)
                preds_mc.append(probs.cpu().numpy()[0])
            
            preds_mc = np.array(preds_mc)
            mean_probs = preds_mc.mean(axis=0)
            std_probs = preds_mc.std(axis=0)
            
            predicted_class = mean_probs.argmax()
            confidence = mean_probs[predicted_class]
            entropy = -np.sum(mean_probs * np.log(mean_probs + 1e-12))
            variation_ratio = 1 - (np.bincount(preds_mc.argmax(axis=1), minlength=mean_probs.shape[0]).max() / n_samples)
            
            results.append({
                "sample_index": idx,
                "true_label": y_test_tensor[idx].item(),
                "predicted_class": predicted_class,
                "confidence": confidence,
                "entropy": entropy,
                "variation_ratio": variation_ratio,
                "mean_probs": mean_probs.tolist(),
                "std_probs": std_probs.tolist()
            })
    
    results_df = pd.DataFrame(results)
    
    # Save summary (without per-class mean/std lists)
    summary_df = results_df.drop(columns=["mean_probs", "std_probs"], errors="ignore")
    summary_df.to_csv(summary_path, index=False)
    results_df.to_csv(detailed_path, index=False)
    
    print(f"✅ Evaluation complete. Test accuracy: {(summary_df['predicted_class'] == summary_df['true_label']).mean():.4f}")
    print(f"✅ Results saved to: {summary_path} (summary), {detailed_path} (detailed)")

    return summary_df, results_df

# Run evaluation + save
summary_df, detailed_df = evaluate_and_save_bbb_test(model, X_test_tensor, y_test_tensor, device,
                                                     n_samples=100,
                                                     summary_path="bbb_summary_test.csv",
                                                     detailed_path="bbb_detailed_test.csv")


✅ Evaluation complete. Test accuracy: 0.9326
✅ Results saved to: bbb_summary_test.csv (summary), bbb_detailed_test.csv (detailed)
